# Телеграм бот 2: функциональность

## Привязка данных к пользователю

Допустим, что мы хотим сделать бота, который запоминает фразу, а потом, по просьбе пользователя, напоминает ее ему. Чтобы решить эту задачу, нам понадобится где-то хранить последнее сообщение пользователя.

Если мы будем использовать переменную, то сможем сохранить сообщение только одного пользователя. Например, Ваня попросил запомнить слово linguist. Мы положили эту строку в переменную note. Потом Василиса попросила запомнить слово elephant, и мы снова положили это в переменную note. Когда Ваня попросит нас напомнить его последнее сообщение, мы напишем ему elephant вместо linguist. Неуспех!

Удобнее всего хранить все данные, которые привязаны к конкретному пользователю, в словаре. Ключем в этом словаре будет id пользователя, а значением - произвольные данные.

Допустим, что наш словарь называется ```notes```. Теперь, когда Ваня (```id1076```) пришлет слово linguist мы положим его в ```notes[1076]```, а слово elephant от Василисы (```id1402```) - в ```notes[1402]```. Так как теперь мы используем разные переменные для хранения слова, сообщения от разных пользователей не будут путаться.

In [2]:
import telebot
import conf

bot = telebot.TeleBot(conf.TOKEN)

#создаем словарь
notes = {}

@bot.message_handler(commands=['remind'])
def remind(message):
    user_id = message.chat.id
    if user_id not in notes:
        bot.send_message(user_id, "Вы мне еще не писали.")
    else:
        bot.send_message(user_id, notes[user_id])

@bot.message_handler(content_types=['text'])
def remember(message):
    user_id = message.chat.id
    notes[user_id] = message.text
    bot.send_message(user_id, "Я запомнил")

bot.polling(none_stop=True)

## Кнопки

Как добавить несколько кнопок к сообщению и реагировать на их нажатия?

In [10]:
import telebot
from telebot import types

import conf

bot = telebot.TeleBot(conf.TOKEN)

# @bot.message_handler(commands=["start"])
# def repeat_all_messages(message):
#     # создаем клавиатуру
#     keyboard = types.InlineKeyboardMarkup()

#     # добавляем на нее две кнопки
#     button1 = types.InlineKeyboardButton(text="Привет! 👋", callback_data="button1")
#     button2 = types.InlineKeyboardButton(text="Кнопка 2", callback_data="button2")
#     keyboard.add(button1)
#     keyboard.add(button2)

#     # отправляем сообщение пользователю
#     bot.send_message(message.chat.id, "Нажмите кнопку!", reply_markup=keyboard)
    

@bot.message_handler(commands=["start"])
def repeat_all_messages(message):
    # создаем клавиатуру
    markup = types.ReplyKeyboardMarkup(row_width=2) # по умолчанию 3
    itembtna = types.KeyboardButton('a')
    itembtnv = types.KeyboardButton('b')
    itembtnc = types.KeyboardButton('c')
    itembtnd = types.KeyboardButton('d')
    itembtne = types.KeyboardButton('e')
    markup.row(itembtna, itembtnv)
    markup.row(itembtnc, itembtnd, itembtne)

    # отправляем сообщение пользователю
    bot.send_message(message.chat.id, "Нажмите кнопку!", reply_markup=markup)

# функция запустится, когда пользователь нажмет на кнопку
@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    if call.message:
        if call.data == "button1":
            bot.send_message(call.message.chat.id, "Вы нажали на первую кнопку.")
        if call.data == "button2":
            bot.send_message(call.message.chat.id, "Вы нажали на вторую кнопку.")

bot.polling(none_stop=True)

## Какие еще есть функции?

In [ ]:
# sendMessage
bot.send_message(chat_id, text)

# editMessageText
bot.edit_message_text(new_text, chat_id, message_id)

# forwardMessage
bot.forward_message(to_chat_id, from_chat_id, message_id)

# All send_xyz functions which can take a file as an argument, can also take a file_id instead of a file.
# sendPhoto
photo = open('/tmp/photo.png', 'rb')
bot.send_photo(chat_id, photo)
bot.send_photo(chat_id, "FILEID")

# sendAudio
audio = open('/tmp/audio.mp3', 'rb')
bot.send_audio(chat_id, audio)
bot.send_audio(chat_id, "FILEID")

## sendAudio with duration, performer and title.
bot.send_audio(CHAT_ID, file_data, 1, 'eternnoir', 'pyTelegram')

# sendVoice
voice = open('/tmp/voice.ogg', 'rb')
bot.send_voice(chat_id, voice)
bot.send_voice(chat_id, "FILEID")

# sendDocument
doc = open('/tmp/file.txt', 'rb')
bot.send_document(chat_id, doc)
bot.send_document(chat_id, "FILEID")

# sendSticker
sti = open('/tmp/sti.webp', 'rb')
bot.send_sticker(chat_id, sti)
bot.send_sticker(chat_id, "FILEID")

# sendVideo
video = open('/tmp/video.mp4', 'rb')
bot.send_video(chat_id, video)
bot.send_video(chat_id, "FILEID")

# sendVideoNote
videonote = open('/tmp/videonote.mp4', 'rb')
bot.send_video_note(chat_id, videonote)
bot.send_video_note(chat_id, "FILEID")

# sendLocation
bot.send_location(chat_id, lat, lon)

# sendChatAction
# action_string can be one of the following strings: 'typing', 'upload_photo', 'record_video', 'upload_video',
# 'record_audio', 'upload_audio', 'upload_document' or 'find_location'.
bot.send_chat_action(chat_id, action_string)

# getFile
# Downloading a file is straightforward
# Returns a File object
import requests
file_info = tb.get_file(file_id)

file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(TOKEN, file_info.file_path))

## Как отправить и как скачать стикер?

Сначала нужно узнать ID стикера. Отправьте любой стикер боту @idstickerbot, и он вернёт Вам ID.

In [16]:
import telebot
from telebot import types

import conf

bot = telebot.TeleBot(conf.TOKEN)


@bot.message_handler(commands=["start"])
def start_reply(message):
    bot.send_message(message.chat.id, "Ну чего тебе?")


@bot.message_handler(content_types=["text"])
def text(message):
    bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEH5Chj-b0srTZNgAG-LsVOrOILfe0d-QACRhAAAuILIEib3ujwu8FH3S4E')
    
bot.polling(none_stop=True)

Если пользователь пришлёт слово sticker, то мы отправим ему стикер. Используем метод send_sticker.

А как сохранить на комп присланный стикер?

In [ ]:
@bot.message_handler(content_types=["sticker"])
def handle_docs_audio(message):
    # получаем ID стикера
    sticker_id = message.sticker.file_id
    # получаем путь, где лежит файл стикера на сервере телеграма
    file_info = bot.get_file(sticker_id)
    # скачиваем файл
    urllib.request.urlretrieve(f'http://api.telegram.org/file/bot{config.token}/{file_info.file_path}', file_info.file_path)

Простой бот, который отвечает текстом или стикером, в зависимости от послания пользователя:

In [21]:
import telebot
import conf 

bot = telebot.TeleBot(conf.TOKEN)

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, ты написал мне, ура!')

@bot.message_handler(content_types=['text'])
def send_text(message):
    print(message.chat.id)
    if message.text.lower() == 'привет':
        bot.send_message(message.chat.id, 'Привет, лингвист!')
    elif message.text.lower() == 'пока':
        bot.send_message(message.chat.id, 'Пока, лингвист!')
    elif message.text.lower() == 'люблю программировать':
        bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEH5Ddj-b9T7Qa3ObWuJ6lNEe0MGmg5oQACRA4AAhT50EudQw4-ewR__y4E')

@bot.message_handler(content_types=['sticker'])
def sticker_id(message):
    print(message)
    
bot.polling()

430675894


## Как отправить длинное текстовое сообщение?

Если Вы хотите отправить сообщение, где больше 5000 символов, его придется разбить

In [29]:
large_text = open("Tolkien J.. The Lord of the Rings - royallib.ru.txt", "rb").read().decode('cp1251')
len(large_text)

2891555

In [28]:
from telebot import util

large_text = open("Tolkien J.. The Lord of the Rings - royallib.ru.txt", "rb").read().decode('cp1251')

# разобьем текст на 3000 символов
# split_string возвращает список разбитых кусочков текста
splitted_text = util.split_string(large_text[:10000], 3000)
for text in splitted_text:
    bot.send_message(430675894, text)

ApiTelegramException: A request to the Telegram API was unsuccessful. Error code: 429. Description: Too Many Requests: retry after 11

## Как обратиться к пользователю по имени?

Объект ```message``` содержит ин-фу о пользователе; чтобы получить имя, нужно использовать ```message.from_user.first_name```

In [ ]:
import telebot
import conf

bot = telebot.TeleBot(conf.token)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет, " + message.from_user.first_name + "!")

bot.polling()

Чтобы узнать больше опций, можно просто распечатать:

In [ ]:
def send_welcome(message):
    print(message.from_user)

## Logging

Можно использовать logger из модуля telebot:

In [ ]:
import logging

logger = telebot.logger
telebot.logger.setLevel(logging.DEBUG) # Outputs debug messages to console.

## Ущё чуть-чуть про ```polling```


- none_stop: True/False (default False) - не прекращай запрашивать, пока не получишь от серверов Телеграма ошибку
- interval: True/False (default False) - интервал между запросами, модификация приводит к замедлению реакции бота 
- timeout: integer (default 20) - промежутки в секундах


In [ ]:
bot.polling(none_stop=False, interval=0, timeout=20)